In [2]:
# From bokeh import the other libraries
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment

palette.reverse()

# Get the list of counties as state (for USA) and filter it for New Jersey
counties = {
    code: county for code, county in counties.items() if county["state"] == "nj"
}

# Get the longitide and latitude
county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

# Get the county names, unemployment rate and color palette accordingly
county_names = [county['name'] for county in counties.values()]
county_rates = [unemployment[county_id] for county_id in counties]
color_mapper = LogColorMapper(palette=palette)

# Set the column data source for longitude, latitude, county name and unemployment rates
source = ColumnDataSource(data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
))

# Get the required tools for the plot. Like pan, wheel zoom, etc.
TOOLS = "pan,wheel_zoom,reset,hover,save"

# Set the plot name. Nothing to provide for x and y axis, and grid line color.
p = figure(
    title="New Jersey Unemployment, 2009", tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None

#Set the plot, plot color and enter the plot data
p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

#Set the other parameters - hover tool, long-lat, etc.
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Unemployment rate)", "@rate%"),
    ("(Long, Lat)", "($x, $y)"),
]

#Display the plot
show(p)